# Advent of Code 2022: Day 12
https://adventofcode.com/2022/day/12


## Part 1
Find the shortest path from the start to the goal

### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Transform the data to a 2d array

In [2]:
data_array2d = []
for line in data_list:
  data_array2d.append(list(line))

### Get a list of lower case letters

In [3]:
height_order = list(map(chr, range(ord('a'), ord('z')+1)))

### Setup helper functions

#### Function to get the elevation at the current position

In [4]:
def getElevation(pos_value):
  # Convert the starting position
  # to "a" and the ending position to "z"
  value = pos_value
  if pos_value == "S":
    value = 'a'
  if pos_value == "E":
    value = 'z'
  return height_order.index(value)

#### Function to find the starting position

In [5]:
def get_start_pos(data):
  nrows = len(data)
  ncols = len(data[0])
  for i in range(nrows):
    for j in range(ncols):
      if data[i][j] == "S":
        return([i, j])

#### Function to find the end position

In [6]:
def get_end_pos(data):
  nrows = len(data)
  ncols = len(data[0])
  for i in range(nrows):
    for j in range(ncols):
      if data[i][j] == "E":
        return([i, j])

#### Function to find the valid neighbors in the current position

In [7]:
def get_neighbors(data, heights, pos, nrows, ncols):
  
  # Get the current position
  # and the corresponding value.
  row = pos[0]
  col = pos[1]
  curr = data[row][col]

  # List of valid neighbors.
  dirs = []
  
  # Get the value from each neighbor
  # if the neighbor exists. Calculate
  # the height difference and add
  # the neighbor to the list if
  # the difference is 1 or less.
  if row > 0:
    new_pos = data[row-1][col]
    up = getElevation(new_pos)-getElevation(curr)
    if up  <=1:
      dirs.append([row-1, col]) 
  
  if row <= nrows-2:
    new_pos = data[row+1][col]
    down = getElevation(new_pos)-getElevation(curr)
    if down <=1:
      dirs.append([row+1, col])
  
  if col > 0:
    new_pos = data[row][col-1]
    left = getElevation(new_pos)-getElevation(curr)
    if left  <=1:
      dirs.append([row, col-1])

  if col <= ncols-2:
    new_pos = data[row][col+1]
    right = getElevation(new_pos)-getElevation(curr)
    if right <=1:
      dirs.append([row, col+1])


  return dirs

### Shortest path function

In [8]:
def find_shortest_path(data, heights, start, end):
  
  # Get number of
  # rows and columns.
  nrows = len(data)
  ncols = len(data[0])

  # Setup necessary lists
  # and dictionaries.

  # List of positions
  # still to explore.
  exploreQ = []
  
  # Dictionary that keeps track of
  # the path by having the current 
  # position as the key and the 
  # previous position as the value.
  pred = {}

  # Dictionary that keeps track of
  # the distance from the start.
  dist = {}

  # List that keeps track of which
  # positions have already been visited.
  visited = []

  # The previous position of the
  # start is itself. The distance
  # from start to start is 0.
  pred[str(start)] = start
  dist[str(start)] = 0

  # Add the starting position
  # to the visited and 
  # exploration lists.
  visited.append(start)
  exploreQ.append(start)

  # The start is not the end.
  foundEnd = (start == end)

  # While there are more
  # positions to explore.
  while(not(not exploreQ) and (not foundEnd)):
    
    # Get the position at the
    # from of the exploration list.
    curr = exploreQ.pop(0)
    
    # Get the valid neighbors
    # for the current position.
    neighbors = get_neighbors(data, heights, curr, nrows, ncols)
    
    # Go through the neighbors.
    for neighbor in neighbors:
      
      # If the neighbor has not
      # already been visited.
      if neighbor not in visited:
        
        # The previous position for
        # this neighbor is the 
        # current position.
        pred[str(neighbor)] = curr
        
        # Add the neighbor position
        # to the visited and
        # exploration lists.
        visited.append(neighbor)
        exploreQ.append(neighbor)
        
        # The distance from the start is 1
        # more than in the previous position.
        if (str(neighbor) not in dist.keys()):
          dist[str(neighbor)] = dist[str(curr)] +1
        
        # Stop the loop if 
        # the end is found.
        if neighbor == end:
          foundEnd = True
          break

  # Create the list for
  # the shortest path.
  path = []

  # The current position is the
  # end, add it to the path.
  curr = str(end)
  path.append(curr)
  
  # While the current position is
  # in the list of all previous positions
  # and is not the same as is not the
  # same as its previous position.
  while((curr in pred.keys()) and (str(pred[curr]) != curr)):
    
    # Insert the previous position
    # of the current position into the list.
    path.insert(0, str(pred[curr]))
    
    # Update the current position to
    # its previous position.
    curr = str(pred[curr])

  return path

### Execute the function

In [9]:
# Get the start and end positions.
start = get_start_pos(data_array2d)
end = get_end_pos(data_array2d)

# Get the shortest path.
p = find_shortest_path(data_array2d, height_order, start, end)

# Subtract by 1 to get the correct value, since
# the starting position is counted as a position
# in the list, but does not contribute to the
# actual path length. 
len(p)-1

504

## Part 2
Find the shortest path from every start of equal height to the end

### Function to find the shortest path with multiple starts

In [10]:
def find_nearest_start(data, heights, startheight, end):
  
  # Get number of
  # rows and columns.
  nrows = len(data)
  ncols = len(data[0])
  
  # Setup a list of starting positions
  # and fill it with positions.
  starts = []
  for i in range(nrows):
    for j in range(ncols):
      if data[i][j] == "S" or data[i][j] == startheight:
        starts.append([i,j])

  # Setup a list of distances,
  # and fill it with the algorithm
  # using 3 as a minimum length.
  dists = []
  for start in starts:
    p = find_shortest_path(data, heights, start, end)
    if len(p) > 3:
      dists.append(len(p)-1)
  return min(dists)

### Execute function

In [11]:
# Get the end position
# and starting height.
end = get_end_pos(data_array2d)
start_height= "a"

# Get the shortest path.
find_nearest_start(data_array2d, height_order, "a", end)

500